In [ ]:
import os 
import pandas as pd 
import plotly.express as px 
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np 


sns.set_theme(style="whitegrid")

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Computer Modern Roman']
plt.rcParams['text.usetex'] = True
plt.rcParams['font.size'] = 8
plt.rcParams['axes.labelsize'] = 8
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 8
markersize = 3
margin_title_size = 8

plot_height = 0.7
line_width = 5.1


from fractions import Fraction
import re


def float_to_latex_fraction(x, limit_denominator=10):
    x = float(re.search(r'\{([^}]+)\}', x.get_text()).group(1).replace('−', '-'))
    frac = Fraction(x).limit_denominator(limit_denominator)
    if frac.denominator == 1:
        return f"${frac.numerator}$"
    return f"$\\frac{{{frac.numerator}}}{{{frac.denominator}}}$"


def set_yticklabels_as_fractions(ax):
    ax.set_yticklabels([float_to_latex_fraction(t) for t in ax.get_yticklabels()])

In [ ]:
results = []
for root, dirs, files in os.walk("results/wind"):
    for file in files:
        if file.endswith(".csv"):
            experiment_name = os.path.basename(root)
            experiment_name = experiment_name.replace("1e-05", "0.00001")
            experiment_name = experiment_name.replace("1e-06", "0.000001")
            experiment_params = experiment_name.split("-")
            experiment_dict = {param.split("=")[0]: param.split("=")[1] for param in experiment_params}

            metrics_path = os.path.join(root, file)
            metrics_dict = pd.read_csv(metrics_path).to_dict(orient="records")[0]

            result_dict = experiment_dict | metrics_dict
            results.append(result_dict)

In [ ]:
data = pd.DataFrame(results).astype(
    {
        'time': int, 
        'level': int, 
        'num_layers': int, 
        'seed': int, 
        'mse': float, 
        'nlpd': float, 
    }
).query((
    "level in (0, 7, 15)"
    "& num_layers in (1, 2, 3, 4, 5)"
    "& seed == 0"
)).sort_values(
    by=["level", "time", "num_layers"]
).replace(
    {
        "level": {0: "5.5 km", 7: "2.0 km", 15: "0.1 km"},
    }
).assign(
    time=lambda df: df.time + 1
).rename(
    columns={
        "level": "Altitude",
        "num_layers": "Number of Layers",
        "mse": "MSE",
        "nlpd": "NLPD", 
        "time": "Month",
    }
)

In [ ]:
g = sns.FacetGrid(
    data,
    col='Altitude',
    hue="Number of Layers", 
)
g.map(sns.lineplot, "Month", "NLPD", marker="o", markersize=markersize)
g.add_legend(
    loc="upper left",
    bbox_to_anchor=(1.0, 0.85),
)
g.legend.set_title("\# Layers", prop={'size': 8})
g.set(xticks=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
g.set_titles(col_template="Altitude $=$ {col_name}")

g.figure.subplots_adjust(left=0.0, right=1.0, top=1.0, bottom=0.0)
g.figure.set_size_inches(line_width / 1.2, plot_height / 1.2 * 2.0)

# plt.savefig("plots/hodge-nlpd_vs_month_num_layers_and_altitude.pdf", bbox_inches='tight')

In [ ]:
g = sns.FacetGrid(
    data,
    col='Altitude',
    hue="Number of Layers", 
)
g.map(sns.lineplot, "Month", "MSE", marker="o", markersize=markersize)
g.add_legend(
    loc="upper left",
    bbox_to_anchor=(1.0, 0.85),
)
g.legend.set_title("\# Layers", prop={'size': 8})
g.set(xticks=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
g.set_titles(col_template="Altitude $=$ {col_name}")

g.figure.subplots_adjust(left=0.0, right=1.0, top=1.0, bottom=0.0)
g.figure.set_size_inches(line_width / 1.2, plot_height / 1.2 * 2.0)

# plt.savefig("plots/hodge-mse_vs_month_num_layers_and_altitude.pdf", bbox_inches='tight')

In [ ]:
errorbar = ('sd', 1)

g = sns.FacetGrid(data, hue="Altitude", margin_titles=True, sharey=False)
g.map(sns.lineplot, "Number of Layers", "NLPD", marker="o", errorbar=errorbar, markersize=markersize)
g.add_legend(
    loc="upper left",
    title="Altitude",
    bbox_to_anchor=(1.0, 1.15),
    ncol=1,
)
g.legend.set_title("Altitude", prop={'size': 8})
g.set(xticks=[1, 2, 3, 4, 5])
g.set_axis_labels("\# Layers")
g.figure.subplots_adjust(left=0.0, right=1.0, top=1.0, bottom=0.0)
g.figure.set_size_inches(line_width / 5.5, plot_height)

for ax in g.axes.flat:
    ax.tick_params(axis='x', pad=-3)
    ax.tick_params(axis='y', pad=-4)


# plt.savefig("plots/hodge-nlpd_vs_num_layers_and_altitude-sd1.pdf", bbox_inches='tight')

In [ ]:
errorbar = ('sd', 1)

g = sns.FacetGrid(data, hue="Altitude", margin_titles=True, sharey=False)
g.map(sns.lineplot, "Number of Layers", "NLPD", marker="o", errorbar=errorbar, markersize=markersize)
g.add_legend(
    loc="upper left",
    title="",
    bbox_to_anchor=(0.05, 1.10),
    ncol=3,
    prop={'size': 8},
    columnspacing=-1.0
)
for txt in g.legend.get_texts():
    txt.set_ha("center") # horizontal alignment of text item
    txt.set_x(-26) # x-position
    txt.set_y(8) # y-position
    
g.set(xticks=[1, 2, 3, 4, 5])
g.set_axis_labels("\# Layers")
g.figure.subplots_adjust(left=0.0, right=1.0, top=0.85, bottom=0.0)
g.figure.set_size_inches(line_width / 3, plot_height * 1.5)

for ax in g.axes.flat:
    ax.tick_params(axis='x', pad=-3)
    ax.tick_params(axis='y', pad=-4)


# plt.savefig("plots/hodge-nlpd_vs_num_layers_and_altitude-sd1-larger.pdf", bbox_inches='tight')

In [ ]:
errorbar = ('sd', 1)

g = sns.FacetGrid(data, hue="Altitude", margin_titles=True, sharey=False)
g.map(sns.lineplot, "Number of Layers", "MSE", marker="o", errorbar=errorbar, markersize=markersize)
g.add_legend(
    loc="upper left",
    title="",
    bbox_to_anchor=(0.05, 1.10),
    ncol=3,
    prop={'size': 8},
    columnspacing=-1.0
)
for txt in g.legend.get_texts():
    txt.set_ha("center") # horizontal alignment of text item
    txt.set_x(-26) # x-position
    txt.set_y(8) # y-position
    
g.set(xticks=[1, 2, 3, 4, 5])
g.set_axis_labels("\# Layers")
g.figure.subplots_adjust(left=0.0, right=1.0, top=0.85, bottom=0.0)
g.figure.set_size_inches(line_width / 3, plot_height * 1.5)

for ax in g.axes.flat:
    ax.tick_params(axis='x', pad=-3)
    ax.tick_params(axis='y', pad=-4)


# plt.savefig("plots/hodge-mse_vs_num_layers_and_altitude-sd1.pdf", bbox_inches='tight')